In [1]:
import pandas as pd
import numpy as np
import os
import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [2]:
raw = pd.read_csv('./data/DS0001/04690-0001-Data.tsv',sep='\t')
raw.shape

(3617, 4564)

In [19]:
df = raw[(raw['V2060'] == 1) & (raw['V2225'] > 0) & (raw['V2017'] > 0)]

missing_val = {-95: np.nan, -96: np.nan, -99: np.nan}
# check_box = {1: 1, 5: 0}
df.replace({'V103': missing_val, 'V104': missing_val, 'V546': missing_val,
            'V1002': missing_val, 'V1006': missing_val, 'V1007': missing_val,
            'V1010': missing_val, 'V1012': missing_val, 'V2007': missing_val,
            'V2017':missing_val, 'V2020': missing_val,
            }, inplace=True)

# Listwise deletion as in the original paper
df = df.dropna(axis='index', subset=['V1002','V1006','V1007','V1010','V1012'], how='any')

# Dependent variables
df['positive_emotion'] = df[['V1006', 'V1010']].mean(axis=1)
df['negative_emotion'] = df[['V1012', 'V1002', 'V1007']].mean(axis=1)

# Normalize
df['positive_emotion']=(df['positive_emotion']-df['positive_emotion'].min()) \
                        /(df['positive_emotion'].max()-df['positive_emotion'].min())
df['negative_emotion']=(df['negative_emotion']-df['negative_emotion'].min()) \
                        /(df['negative_emotion'].max()-df['negative_emotion'].min())

# Independent variables
df['spouse_support'] = df['V2204']
df['spouse_strain'] = df['V2205']
df['child_support'] = df['V2207']
df['child_strain'] = df['V2208']
df['friend_support'] = df['V2216']
df['friend_strain'] = df['V2217']

def age_categorize(row):  
    if row['age'] < 45:
        return 'under 45'
    elif row['age'] >= 45 and row['age'] <= 65:
        return '45-65'
    elif row['age'] > 65:
        return 'over 65'

def education_categorize(row):  
    if row['education'] < 12:
        return '1'
    elif 12 <= row['education'] < 14:
        return '2'
    elif 14 <= row['education'] < 16:
        return '3'
    elif 16 <= row['education'] < 17:
        return '4'
    elif row['education'] >= 17:
        return '5'

# Control variables
df['confidants'] = df['V546']
df['age'] = df['V104']
df['income'] = df['V2020']
df['sex'] = df['V103']
df['education'] = df['V2007']
df['retired'] = df['V1105']
df['num_child'] = df['V2017']

df = df.dropna(axis='index', subset=['confidants','age','age','income','sex',
                                    'education','retired','num_child'], how='any')

df['age_group'] = df.apply(lambda row: age_categorize(row), axis=1)
df['education_group'] = df.apply(lambda row: education_categorize(row), axis=1)

## Dataset sampling

- Originally 3617 respondents
- Listwise deletion to 1640 participants
- Use dropna() function for Listwise deletion
- All of these participants had a spouse, a child, and friends when responding to the survey

In [3]:
# Married, have friends
# V2060: Married == 1
# V2225: # of Friends > 0
# V2017: # of Children > 0

df = raw[(raw['V2060'] == 1) & (raw['V2225'] > 0) & (raw['V2017'] > 0)]
print(len(df))

1768


In [4]:
df.isnull().values.any()

False

## Dependent Variables

Positive emotional state

- “In the past week: I felt happy” - H1E V1006
- “In the past week: I enjoyed life” - H1J V1010
- The two variables are combined to V2620


Negative emotional state
- “In the past week: I felt sad” - H1M V1012
- “In the past week: I felt depressed” - H1A V1002
- “In the past week: I felt lonely” - H1F V1007

In [5]:
missing_val = {-95: np.nan, -96: np.nan, -99: np.nan}

df.replace({'V1006': missing_val, 'V1010': missing_val, 'V1012': missing_val, 
            'V1002': missing_val, 'V1007': missing_val}, inplace=True)

df = df.dropna(axis='index', subset=['V1002','V1006','V1007','V1010','V1012'], how='any')


df['positive_emotion'] = df[['V1006', 'V1010']].mean(axis=1)
df['negative_emotion'] = df[['V1012', 'V1002', 'V1007']].mean(axis=1)

# Normalize
df['positive_emotion']=(df['positive_emotion']-df['positive_emotion'].min()) \
                    /(df['positive_emotion'].max()-df['positive_emotion'].min())
df['negative_emotion']=(df['negative_emotion']-df['negative_emotion'].min()) \
                    /(df['negative_emotion'].max()-df['negative_emotion'].min())
print(len(df))

1740


## Independent Variables

Spouse
- Love and care: C2 V405
- Willing to listen: C4 V407
- Too many demands: C3 V406
- Critical of you: C5 V408

Child
- Love and care: C14 V431
- Willing to listen: C16 V433
- Too many demands: C15 V432
- Critical of you: C17 V434

Friends
- Love and care: C47 V535
- Willing to listen: C49 V537
- Too many demands: C48 V536
- Critical of you: C50 V538

In [6]:
df['spouse_support'] = df['V2204']
df['spouse_strain'] = df['V2205']
df['child_support'] = df['V2207']
df['child_strain'] = df['V2208']
df['friend_support'] = df['V2216']
df['friend_strain'] = df['V2217']

In [7]:
df.isnull().values.any()

False

In [8]:
len(df)

1740

## Control Variables

- Confidants: C54 V546
- Age: A0C1 V104
- Income: R27 V1730
- Sex: AOB1 V103
- Education R13 V1646
    - Less than HS
    - HS diploma
    - College degree
    - 4-year degree
    - Graduate degree
- Retired: J1E V1105
- Age category
    - Age <45
    - Age 45-65
    - Age >65
- Number of Children
    - Children in HH: V105, V108, V111, V114, V117, V120, V123, V126, V129, V132, V135 -> Check each column 
    - Children elsewhere: V202



In [18]:
data['age_group'].head()

KeyError: 'age_group'

In [10]:
def age_categorize(row):  
    if row['age'] < 45:
        return 'under 45'
    elif row['age'] >= 45 and row['age'] <= 65:
        return '45-65'
    elif row['age'] > 65:
        return 'over 65'
    
def education_categorize(row):  
    if row['education'] < 12:
        return '1' # Less than high school
    elif 12 <= row['education'] < 14:
        return '2' # High school diploma
    elif 14 <= row['education'] < 16:
        return '3' # College degree
    elif 16 <= row['education'] < 17:
        return '4' # 4-year degree
    elif row['education'] >= 17:
        return '5' # Graduate degree
#     return np.nan

In [15]:
data['education_group'].head()

1     2
9     1
11    1
12    3
13    1
Name: education_group, dtype: object

In [11]:
missing_val = {-95: np.nan, -96: np.nan, -99: np.nan}
check_box = {1: 1, 5: 0}

df.replace({'V546': missing_val, 'V104': missing_val, 'V2020': missing_val,
            'V1105': check_box, 'V103': missing_val, 'V2007': missing_val,
            'V2017':missing_val
            }, inplace=True)

df['confidants'] = df['V546']
df['age'] = df['V104']
# df['age_group'] = df.apply(lambda row: age_categorize(row), axis=1)
df['income'] = df['V2020']
df['sex'] = df['V103']
df['education'] = df['V2007']
# df['education_group'] = df.apply(lambda row: education_categorize(row), axis=1)
df['retired'] = df['V1105']
df['num_child'] = df['V2017']

df = df.dropna(axis='index', subset=['confidants','age','age','income','sex',
                                     'education','retired','num_child'], how='any')

# df['age_group'] = df.apply(lambda row: age_categorize(row), axis=1)
# df['education_group'] = df.apply(lambda row: education_categorize(row), axis=1)

In [12]:
df.isnull().values.any()

False

In [13]:
len(df)

1585

In [15]:
# data = df[['positive_emotion', 'negative_emotion', 'spouse_support', 'spouse_strain',
#            'child_support', 'child_strain', 'friend_support', 'friend_strain',
#            'confidants', 'age', 'age_group', 'income', 'sex', 'education', 'education_group',
#            'retired', 'num_child']].copy(deep=True)

data = df[['positive_emotion', 'negative_emotion', 'spouse_support', 'spouse_strain',
           'child_support', 'child_strain', 'friend_support', 'friend_strain',
           'confidants', 'age', 'income', 'sex', 'education',
           'retired', 'num_child']].copy(deep=True)

print(data.shape)
data.head()

(1585, 15)


,positive_emotion,negative_emotion,spouse_support,spouse_strain,child_support,child_strain,friend_support,friend_strain,confidants,age,income,sex,education,retired,num_child
1,0.5,0.000000,0.3309,1.3306,-0.4619,1.814400,-2.045698,0.3005,1.0,44.0,9,1,12,0,2
9,1.0,0.333333,0.3309,-1.3283,-0.1654,-0.442000,0.114700,0.2565,4.0,73.0,2,1,9,1,3
11,0.5,0.166667,-0.4141,-0.7946,-0.1654,-0.482300,-0.401300,0.9053,3.0,47.0,7,1,10,0,5
12,1.0,0.000000,0.8749,-0.2641,0.9599,-1.026199,1.243099,-0.9531,2.0,48.0,7,1,14,1,3
13,1.0,0.166667,0.3309,-0.7946,0.9599,-1.026199,-0.497800,1.4661,7.0,55.0,7,2,5,0,3


## Statistical Hybrid Model

In [16]:
len(data)

1585

In [30]:
df = raw[(raw['V2060'] == 1) & (raw['V2225'] > 0) & (raw['V2017'] > 0)]

missing_val = {-95: np.nan, -96: np.nan, -99: np.nan}
# check_box = {1: 1, 5: 0}
df.replace({'V103': missing_val, 'V104': missing_val, 'V546': missing_val,
            'V1002': missing_val, 'V1006': missing_val, 'V1007': missing_val,
            'V1010': missing_val, 'V1012': missing_val, 'V2007': missing_val,
            'V2017':missing_val, 'V2020': missing_val,
            }, inplace=True)

# Listwise deletion as in the original paper
df = df.dropna(axis='index', subset=['V1002','V1006','V1007','V1010','V1012'], how='any')

# Dependent variables
df['positive_emotion'] = df[['V1006', 'V1010']].mean(axis=1)
df['negative_emotion'] = df[['V1012', 'V1002', 'V1007']].mean(axis=1)

# Normalize
df['positive_emotion']=(df['positive_emotion']-df['positive_emotion'].min()) \
                        /(df['positive_emotion'].max()-df['positive_emotion'].min())
df['negative_emotion']=(df['negative_emotion']-df['negative_emotion'].min()) \
                        /(df['negative_emotion'].max()-df['negative_emotion'].min())

# Independent variables
df['spouse_support'] = df['V2204']
df['spouse_strain'] = df['V2205']
df['child_support'] = df['V2207']
df['child_strain'] = df['V2208']
df['friend_support'] = df['V2216']
df['friend_strain'] = df['V2217']

def age_categorize(row):  
    if row['age'] < 45:
        return '0'
    elif row['age'] >= 45 and row['age'] <= 65:
        return '1'
    elif row['age'] > 65:
        return '2'

def education_categorize(row):  
    if row['education'] < 12:
        return '0'
    elif 12 <= row['education'] < 14:
        return '1'
    elif 14 <= row['education'] < 16:
        return '2'
    elif 16 <= row['education'] < 17:
        return '3'
    elif row['education'] >= 17:
        return '4'

# Control variables
df['confidants'] = df['V546']
df['age'] = df['V104']
df['income'] = df['V2020']
df['sex'] = df['V103']
df['education'] = df['V2007']
df['retired'] = df['V1105']
df['num_child'] = df['V2017']

df = df.dropna(axis='index', subset=['confidants','age','age','income','sex',
                                    'education','retired','num_child'], how='any')

df['age_category'] = df.apply(lambda row: age_categorize(row), axis=1)
df['education_group'] = df.apply(lambda row: education_categorize(row), axis=1)

In [41]:
from statsmodels.regression.mixed_linear_model import MixedLM

# For now, this is what we can do.
# In fact, this is what they do!
# data = data.dropna()

# Describe variance components
vc = {'confidants': '0 + C(confidants)', 'age_category': '0 + C(age_category)', 'education_group': '0 + C(education_group)', 
      'income': '0 + C(income)', 'sex': '0 + C(sex)', 'retired': '0 + C(retired)', 'num_child': '0 + C(num_child)'} 

model1 = MixedLM.from_formula(
    'positive_emotion ~ spouse_support + spouse_strain + child_support + child_strain + friend_support + friend_strain',    
    vc_formula = vc, data=df, groups=df['age_category'])

#     'positive_emotion ~ spouse_support + spouse_strain + child_support + child_strain + friend_support + friend_strain',
#     vc_formula = vc, data=data, groups=data['age_group'])

result1 = model1.fit()
result1.summary()


/opt/miniconda3/envs/work/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/work/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/opt/miniconda3/envs/work/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/opt/miniconda3/envs/work/lib/python3.10/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(
/opt/miniconda3/envs/work/lib/python3.10/site-packages/statsmodels/base/model.py:604: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Che

<class 'statsmodels.iolib.summary2.Summary'>
"""
            Mixed Linear Model Regression Results
==============================================================
Model:            MixedLM Dependent Variable: positive_emotion
No. Observations: 1585    Method:             REML            
No. Groups:       3       Scale:              0.0521          
Min. group size:  378     Log-Likelihood:     3.8430          
Max. group size:  621     Converged:          No              
Mean group size:  528.3                                       
--------------------------------------------------------------
                    Coef.  Std.Err.   z    P>|z| [0.025 0.975]
--------------------------------------------------------------
Intercept            0.856    0.171  5.016 0.000  0.521  1.190
spouse_support       0.032    0.006  5.001 0.000  0.020  0.045
spouse_strain       -0.011    0.007 -1.618 0.106 -0.024  0.002
child_support        0.017    0.005  3.330 0.001  0.007  0.027
child_strain        -0.017    0.005 -3.317 0.001 -0.027 -0.007
friend_support       0.020    0.007  2.845 0.004  0.006  0.034
friend_strain       -0.011    0.007 -1.639 0.101 -0.025  0.002
age_category Var     0.048                                    
confidants Var       0.000                                    
education_group Var  0.007                                    
income Var           0.005                                    
num_child Var        0.000                                    
retired Var          0.036                                    
sex Var              0.036                                    
==============================================================

"""

In [37]:
from statsmodels.regression.mixed_linear_model import MixedLM

# For now, this is what we can do.
# In fact, this is what they do!
# data = data.dropna()

# Describe variance components
vc = {'confidants': '0 + C(confidants)', 'age_category': '0 + C(age_category)', 'education_group': '0 + C(education_group)', 
      'income': '0 + C(income)', 'sex': '0 + C(sex)', 'retired': '0 + C(retired)', 'num_child': '0 + C(num_child)'} 

model2 = MixedLM.from_formula(
    'negative_emotion ~ spouse_support + spouse_strain + child_support + child_strain + friend_support + friend_strain \
        + C(confidants) + C(sex, Treatment(reference="0") + C(income) + C(education_group, Treatment(reference="0")) \
        + C(age_category) + C(retired) + C(num_child)',    
    vc_formula = vc, data=df, groups=df['age_category'])

result2 = model2.fit()
result2.summary()


PatsyError: unclosed bracket in embedded Python expression
    negative_emotion ~ spouse_support + spouse_strain + child_support + child_strain + friend_support + friend_strain         + C(confidants) + C(sex, Treatment(reference="0") + C(income) + C(education_group, Treatment(reference="0"))         + C(age_category) + C(retired) + C(num_child)
                                                                                                                                                ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [21]:
df.to_pickle("./pierce2019who_dataframe.pickle") 